In [2]:
import numpy as np 
import tensorflow as tf 
import keras
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Flatten, Dropout
from keras.layers import BatchNormalization, LSTM

import string
from preprocessor import load_sonnets

sonnets = load_sonnets()

In [3]:
chars = list(string.ascii_lowercase)
#chars = list(string.ascii_letters)
chars += [",", ".", "?", "!", ":", ";", "'", "(", ")", " ", "\n", "-"]

char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

train_strings = []
char_length = 41
char_skip = 3 #20

for _, s in enumerate(sonnets):
    #sonnet = "\n".join(sonnets[0]).lower()
    sonnet = "\n".join(s).lower()
    
    for i in range(0, len(sonnet) - char_length, char_skip):
        train_strings.append(sonnet[i: i+char_length])
    #break
    
x = np.zeros((len(train_strings), char_length - 1, len(chars)), dtype=np.bool)
y = np.zeros((len(train_strings), len(chars)), dtype=np.bool)

for i, train_string in enumerate(train_strings):
    for t, char in enumerate(train_string[:-1]):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[train_string[-1]]] = 1

In [4]:
#timesteps = 8
model = Sequential()
#model.add(Dense(200, input_shape=(200,)))
model.add(LSTM(200, input_shape=(char_length - 1, len(chars))))
model.add(Dense(200))
model.add(Activation('relu'))
model.add(Dropout(0.3))

model.add(Dense(len(chars)))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

In [5]:
#model.summary()
model.fit(x, y, batch_size=64, epochs=30)

Epoch 1/30
29325/29325 [==============================] - 18s 600us/step - loss: 2.7539 - acc: 0.2263
Epoch 2/30
29325/29325 [==============================] - 17s 571us/step - loss: 2.2441 - acc: 0.3426
Epoch 3/30
29325/29325 [==============================] - 17s 573us/step - loss: 2.0696 - acc: 0.3898
Epoch 4/30
29325/29325 [==============================] - 17s 591us/step - loss: 1.9668 - acc: 0.4167
Epoch 5/30
29325/29325 [==============================] - 17s 586us/step - loss: 1.8798 - acc: 0.4376
Epoch 6/30
29325/29325 [==============================] - 17s 574us/step - loss: 1.8101 - acc: 0.4537
Epoch 7/30
29325/29325 [==============================] - 17s 576us/step - loss: 1.7413 - acc: 0.4711
Epoch 8/30
29325/29325 [==============================] - 17s 568us/step - loss: 1.6882 - acc: 0.4845
Epoch 9/30
29325/29325 [==============================] - 17s 568us/step - loss: 1.6337 - acc: 0.4977
Epoch 10/30
29325/29325 [==============================] - 17s 566us/step - loss: 

In [6]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [7]:
def generate_poem(seed = "shall i compare thee to a summer's day?\n", lines=14):
    generated = "" + seed

    # Generates 14 lines of poem
    
    for i in range(lines):
        while True:
            x_pred = np.zeros((1, char_length - 1, len(chars)))
            for t, char in enumerate(seed):
                x_pred[0, t, char_indices[char]] = 1.
            preds = model.predict(x_pred, verbose=0)[0]
        
            next_index = sample(preds, temperature=0.5)
            next_char = indices_char[next_index]
            seed = seed[1:] + next_char
        
            generated += next_char
            if next_char == "\n":
                #print(generated)
                #generated = ""
                break
            #generated += next_char

    return generated
    
print(generate_poem())

/home/nmeyer/Documents/cs155/project3/tensorflow/lib64/python3.6/site-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log
  after removing the cwd from sys.path.


shall i compare thee to a summer's day?
the whry of her you with thy living thee,
o thy self thy self aw reserved to show,
they douncessed excue the know my heart in the.
o must leave and way thy beauty thou mood,
  and my heart thee thoughts, or why seffece,
when i (my sone but love, on the ward,
and the fimmert stand in heart's still,
for anour and sweet repubing that he vee.
f the wart not be thou love thy will,
and beauty's lies and the gard to making,
and that i mise see self-tiandot wanderest love,
a moning tan thee i my love thee relity,
no reast that with each sweet reloved,
  maning so mresink can beas not came, decrowitk's vile,



In [8]:
#seed = "shall i compare thee to a summer's day?\nthou art more lovely"
seed = "shall i compare thee to a summer's day?\n"
generated = "" + seed

# Generates 14 lines of poem (More efficient)

x_pred = np.zeros((1, char_length - 1, len(chars)))
for t, char in enumerate(seed):
    x_pred[0, t, char_indices[char]] = 1.
preds = model.predict(x_pred, verbose=0)[0]

for i in range(14):
    while True:
        next_index = sample(preds, temperature=0.5)
        next_char = indices_char[next_index]
        
        x_pred[0, :-1, :] = x_pred[:, 1:, :]
        x_pred[0, -1, :] = np.zeros(len(chars))
        x_pred[0, -1, char_indices[next_char]] = 1.
        preds = model.predict(x_pred, verbose=0)[0]
        
        generated += next_char
        if next_char == "\n":
            #print(generated)
            #generated = ""
            break
        #generated += next_char
        

print(generated)

/home/nmeyer/Documents/cs155/project3/tensorflow/lib64/python3.6/site-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log
  after removing the cwd from sys.path.


shall i compare thee to a summer's day?
thy sweal see to me sure, and the wear weel,
  and in purse,
one pitterne make than blage recore:
the ward the plyis of you wist me self thy well,
that i (our so love, nor thy will be thee,
  you hastur thin, it see and troamere of my seath,
which then desire, the well rary thee there,
the see thy self thy menure of mearth's my days my say,
even wither belonced tongursed of proand,
and many heart beloved to bust, all be,
but in the beauty that my hade on me thath,
and sienct my frouthes in thy self to sigut,
and make as thine or thine, he vounst of thee,
  that dishervel noth which in a sen thee,



In [23]:
print(y.shape)
#print(generated)

(29325, 38)

